In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Local VectorDB

In [68]:
import storage.client

client = storage.client.create_local_db(remove_if_exists=True)

In [69]:
import storage.collection


img_dim = 2
img_similiary_metric = "COSINE"
img_collection = storage.collection.ImageCollection(client, dimension=img_dim, metric_type=img_similiary_metric)

# Insert with Dummy Embeddings

In [70]:

import storage.dataset


dataset = storage.dataset.Dataset(img_folder_path="./data/img_align_celeba",
                        img_identity_map_path="./data/identity_CelebA.txt")

img_identity_collection = dataset.images_by_identity()

img_identity_collection

{2880: ['./data/img_align_celeba/000001.jpg',
  './data/img_align_celeba/000404.jpg',
  './data/img_align_celeba/003415.jpg',
  './data/img_align_celeba/004390.jpg',
  './data/img_align_celeba/018062.jpg',
  './data/img_align_celeba/025244.jpg',
  './data/img_align_celeba/027771.jpg',
  './data/img_align_celeba/039393.jpg',
  './data/img_align_celeba/047978.jpg',
  './data/img_align_celeba/049142.jpg',
  './data/img_align_celeba/052385.jpg',
  './data/img_align_celeba/052623.jpg',
  './data/img_align_celeba/053184.jpg',
  './data/img_align_celeba/053311.jpg',
  './data/img_align_celeba/055834.jpg',
  './data/img_align_celeba/058188.jpg',
  './data/img_align_celeba/061431.jpg',
  './data/img_align_celeba/068154.jpg',
  './data/img_align_celeba/084705.jpg',
  './data/img_align_celeba/090937.jpg',
  './data/img_align_celeba/096324.jpg',
  './data/img_align_celeba/100990.jpg',
  './data/img_align_celeba/103728.jpg',
  './data/img_align_celeba/108341.jpg',
  './data/img_align_celeba/110376.

In [71]:
id1_images = img_identity_collection[1]
id2_images = img_identity_collection[2]

print(f"# Celebrity-ID 1  #img={len(id1_images)}")
print(f"# Celebrity-ID 2  #img={len(id2_images)}")

# Celebrity-ID 1  #img=29
# Celebrity-ID 2  #img=8


In [72]:
# TODO: DUMMY Embedding replace with real img2vec embedding!
import numpy as np

np.random.seed(7)

def embedding_celebrity_id1(img_path: str, dim: int):
    vector = np.random.normal(size=dim, loc=[100, 100], scale=10)
    return vector

def embedding_celebrity_id2(img_path: str, dim: int):
    vector = np.random.normal(size=dim, loc=[-100, 30], scale=50)
    return vector

In [73]:
# TODO: dummy embeddings for the first three images of celebrities 1 and 2
dummy_data = []

dummy_data.extend([{'celeb_id': 1, 'img_path': img_path, 'vector': embedding_celebrity_id1(img_path, dim=img_dim) }
                   for img_path in id1_images[:3]])
dummy_data.extend([{'celeb_id': 2, 'img_path': img_path, 'vector': embedding_celebrity_id2(img_path, dim=img_dim) }
                   for img_path in id2_images[:3]])
dummy_data

[{'celeb_id': 1,
  'img_path': './data/img_align_celeba/000023.jpg',
  'vector': array([116.90525704,  95.34062629])},
 {'celeb_id': 1,
  'img_path': './data/img_align_celeba/004506.jpg',
  'vector': array([100.32820164, 104.07516283])},
 {'celeb_id': 1,
  'img_path': './data/img_align_celeba/006439.jpg',
  'vector': array([ 92.11076971, 100.02065573])},
 {'celeb_id': 2,
  'img_path': './data/img_align_celeba/016188.jpg',
  'vector': array([-100.04451929,  -57.73621532])},
 {'celeb_id': 2,
  'img_path': './data/img_align_celeba/051523.jpg',
  'vector': array([-49.11709972,  60.0249258 ])},
 {'celeb_id': 2,
  'img_path': './data/img_align_celeba/068490.jpg',
  'vector': array([-131.2714487 ,   21.42258694])}]

In [74]:
img_collection.insert(dummy_data)

E20250101 10:48:58.483577 144623 collection_data.cpp:84] [SERVER][Insert][] Insert data failed, errs: attempt to write a readonly database


{'insert_count': 6, 'ids': [455009878273687600, 455009878273687601, 455009878273687602, 455009878273687603, 455009878273687604, 455009878273687605], 'cost': 0}

# Query Vector DB

In [75]:
# Celeb-ID 1 dummy query vector embedding
query_vector = embedding_celebrity_id1("uploaded_image.jpg", img_dim)
query_vector

array([105.05299374,  97.38643585])

In [76]:
# output max 2 entries
result = img_collection.search(query_vectors=[query_vector], limit=2)

for hits in result:
    for hit in hits:
        hit_id, hit_distance = hit['id'], hit['distance']
        print(f"{hit_id=}, {hit_distance=}")

        celeb_id, img_path = hit['entity']['celeb_id'], hit['entity']['img_path']
        print(f" > celeb_id={celeb_id}")
        print(f" > img_path={img_path}")
        print()

hit_id=455009625439469572, hit_distance=0.9998276829719543
 > celeb_id=2
 > img_path=./data/img_align_celeba/051523.jpg

hit_id=455009625439469569, hit_distance=0.9984222054481506
 > celeb_id=1
 > img_path=./data/img_align_celeba/004506.jpg

